In [1]:
import numpy as np 
import pandas as pd

from subprocess import check_output



이 커널은 특히 Keras를 사용하여 CNN 구축을 실험하려는 초보자를 위한 것입니다. 
이 커널을 사용하면 좋은 점수를 얻고 케라스를 배울 수 있습니다. Keras는 모델을 초기화하고 원하는 레이어를 계속 쌓을 수있는 간단한 프레임 워크입니다. 
심층 신경망을 매우 쉽게 구축 할 수 있습니다.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [5]:
# json파일을 불러오기 위해 pd.read_json을 사용한다.
#Load the data.
train = pd.read_json("./input/train.json")

print(train.head())

         id                                             band_1  \
0  dfd5f913  [-27.878360999999998, -27.15416, -28.668615, -...   
1  e25388fd  [-12.242375, -14.920304999999999, -14.920363, ...   
2  58b2aaa0  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  4cfc3a18  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  271f93f4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2 inc_angle  is_iceberg  
0  [-27.154118, -29.537888, -31.0306, -32.190483,...   43.9239           0  
1  [-31.506321, -27.984554, -26.645678, -23.76760...   38.1562           0  
2  [-24.870956, -24.092632, -20.653963, -19.41104...   45.2859           1  
3  [-27.889421, -27.519794, -27.165262, -29.10350...   43.8306           0  
4  [-27.206915, -30.259186, -30.259186, -23.16495...   35.6256           0  


In [23]:
test = pd.read_json("./input/test.json")

주어진 이미지가 빙하인지 배인지 구분하는 문제.

feature로는 band_1, band_2, inc_angle이 있음.

target = is_iceberg
1 = iceberg
0 = ship

위성 사진 : 위성에서 쏜 radar가 특정 object를 만나면 bounce하여 다시돌아오는데 이걸 image로 저장한 것.
이 때 object가 solid할 수록(land, islands, sea ice, icebergs, ships) 더 강한 rader energy가 reflect 되며, 이 에너지를 backscatter이라 함.

이 backscatter는 주변 환경에 영향을 받는데 주변에 바람이 강할 수록 이미지가 밝아지고, 바람이 약할 수록 이미지가 어두워짐. 


지금 주어진 이미지를 찍는 위성은 Sentinel-1인데 측방 감시 레이더. 
즉 특정 angle로 이미지를 보는 것.

주어진 inc_angle은 band_1((이미지), band_2(이미지)를 바라보는 angle. 
(일반적으로 높은 incidence angle일 수록 ocean background가 어두워진다고 함)

band_1, band_2는 이미지. incident_angle은 band_1, band_2(이미지)를 바라보는 각도(즉, band_1, band_2는 상대적인 이미지임)


band_1과 band_2는 HH와 HV로 얻어진 레이더 데이터를 정제하여 얻은 계수, 이미지로 생각하면 됨. 
band_1은  HH(transmit/receive horizontally)
band_2는 HV(transmit horizontally, receive vertically)로 얻어진 레이더 데이터를 정제하여 얻은 것.



본 대회의 데이터는 영상 정보로 위성 레이더 값을 사용하기 때문에 이미지에 해당하는 데이터가 기존 RGB값처럼 3채널이 아니라

HH(transmit/receive horizontally) 와 HV(transmit horizontally, receive vertically)라 불리는 2채널로 이루어져 있고 보편적인 8비트 이미지(0~255사이 정수값)이 아니라 값이 소수점을 가짐

a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])




In [12]:
# training data 를 만듬
# HH, HV, avg를 가진 3개의 밴드를 만듬.
# band_1과 band_2를 75 x 75로 바꾸고, 2D CNN 학습을 위하여 [N, height, width, channel]의 차원을 가지게 바꿈.

X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])

X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

print(X_band_1.shape)
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], 
                          X_band_2[:, :, :, np.newaxis], 
                          ((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

print(X_train.shape)
print(X_train)

(1604, 75, 75)
(1604, 75, 75, 3)
[[[[-27.87836  -27.154118 -27.51624 ]
   [-27.15416  -29.537888 -28.346024]
   [-28.668615 -31.0306   -29.849607]
   ...
   [-25.573483 -29.095308 -27.334396]
   [-26.488674 -29.540878 -28.014776]
   [-30.507013 -27.511766 -29.00939 ]]

  [[-28.66853  -31.030518 -29.849524]
   [-27.8784   -33.529335 -30.703869]
   [-27.508776 -31.591175 -29.549976]
   ...
   [-26.488632 -34.288055 -30.388344]
   [-28.671562 -28.267496 -28.469528]
   [-31.594166 -26.488716 -29.041441]]

  [[-28.66853  -33.52929  -31.09891 ]
   [-27.15416  -37.05116  -32.10266 ]
   [-25.865042 -31.591175 -28.728107]
   ...
   [-27.157106 -31.594084 -29.375595]
   [-27.881393 -32.19339  -30.037392]
   [-32.837124 -29.54092  -31.189022]]

  ...

  [[-29.092905 -27.87895  -28.485928]
   [-28.669163 -27.878992 -28.274078]
   [-30.504612 -30.00814  -30.256376]
   ...
   [-30.011005 -25.28922  -27.650112]
   [-25.868538 -27.512272 -26.690405]
   [-26.489265 -28.672153 -27.58071 ]]

  [[-31.5916

In [15]:

#Take a look at a iceberg
import plotly.offline as py
import plotly.graph_obj as go
py.init_notebook_mode(connected=True)

def plotmy3d(c, name):
    data = [
        go.Surface(
            z=c
        )
    ]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=700,
        height=700,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
plotmy3d(X_band_1[12, :, :], 'iceberg')

ModuleNotFoundError: No module named 'plotly.graph_obj'

이 커널은 특히 Keras를 사용하여 CNN 구축을 실험하려는 초보자를위한 것입니다. 
이 커널을 사용하면 좋은 점수를 얻고 케라스를 배울 수 있습니다. 
Keras는 모델을 초기화하고 원하는 레이어를 계속 쌓을 수있는 간단한 프레임 워크입니다. 
심층 신경망을 매우 쉽게 구축 할 수 있습니다.


우리가 가진 빙산은 멋져 보인다. 레이더 데이터에서 빙산의 모양은 여기와 같이 산과 같을 것이라는 것을 기억하라. 이것은 실제 이미지는 아니지만 레이더에서 산란되기 때문에 모양은 이와 같이 봉우리와 왜곡을 갖게 될 것이다. 배의 모양은 점과 같을 것이고, 길게 뻗은 점과 같을 것이다. 

여기서부터 구조적 차이가 발생하고 우리는 CNN을 이용하여 그 차이점을 이용할 수 있다. 레이더의 백스캐터를 이용해 합성 영상을 만들 수 있다면 도움이 될 것이다.

In [ ]:
plotmy3d(X_band_1[14,:,:], 'Ship')

그것은 배입니다. 길쭉한 점처럼 보입니다. 
우리는 우주선의 모양을 시각화하기 위해 이미지의 해상도가별로 없습니다. 
그러나 CNN이 도와 드리겠습니다. 선박 빙산 분류에 대한 다음과 같은 논문은 거의 없습니다 : 
http://elib.dlr.de/99079/2/2016_BENTES_Frost_Velotto_Tings_EUSAR_FP.pdf 그러나 그들의 데이터는 훨씬 더 나은 해상도를 가지므로 그들이 사용하는 CNN이 여기에 적합하다고 생각하지 않습니다 .


**** Keras를 사용하여 CNN 구축으로 돌아갑니다. 다른 프레임 워크보다 훨씬 나은 프레임 워크. 당신은 확실히 즐길 것입니다.

그것은 배입니다. 길쭉한 점처럼 보입니다. 우리는 우주선의 모양을 시각화하기 위해 이미지의 해상도가별로 없습니다. 그러나 CNN이 도와 드리겠습니다. 선박 빙산 분류에 대한 다음과 같은 논문은 거의 없습니다. .

Keras를 사용하여 CNN 구축으로 돌아가십시오. 다른 프레임 워크보다 훨씬 나은 프레임 워크. 당신은 확실히 즐길 것입니다.

In [18]:
#import  Keras
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

ModuleNotFoundError: No module named 'keras'

In [ ]:
# training data 만들기
# HH, HV 와 둘 다에 대한 평균값을 가진 3개의 BAND 만들기

X_band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

In [10]:
# take a look at iceberg

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(conneted=True)

def plotmy3d(c, name):
    
    data = [
        go.Surface(
            z=c
        )
    ]
    layout = go.Layout(
    )

TypeError: init_notebook_mode() got an unexpected keyword argument 'conneted'

In [9]:


submission = pd.DataFrame()
submission['id'] = test['id']
submission['is_iceberg'] = predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)

NameError: name 'predicted_test' is not defined

결론
점수를 높이기 위해 Speckle 필터링, Indicence angle normalization 및 기타 전처리를 시도했지만 작동하지 않는 것 같습니다. 당신은 시도하고 볼 수 있지만 나에게 그들은 좋은 결과를주지 않습니다.

이 커널을 사용하여 10 위를 차지할 수 없으므로 여기에 아름다운 정보가 있습니다. 테스트 데이터 세트에는 8000 개의 이미지가 포함되어 있습니다. 의사 라벨링을 수행하여 예측을 높일 수 있습니다. 여기에 관련 기사가 있습니다 : https://towardsdatascience.com/simple-explanation-of-semi-supervised-learning-and-pseudo-labeling-c2218e8c769b

이 커널이 마음에 들면 Upvote.